In [5]:
import cv2
import numpy as np
import math as m

In [6]:
cv2.__version__

'4.5.5'

In [7]:
image=cv2.imread('InputImages/Petro - copia (4).jpg')
width, height = image.shape[:2]
img2 = cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
cv2.imshow('imagen',img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
Kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
Kernel1=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
Kernel2=cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3))

In [9]:
gris=cv2.cvtColor(img2,cv2.COLOR_RGB2GRAY)

cv2.imshow('gris',gris)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
equ=cv2.equalizeHist(gris)
cv2.imshow('ecuailzada',equ)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
cierre=cv2.erode(equ,Kernel2,iterations=1)
cv2.imshow('cierre',cierre)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
Bordes=cv2.Canny(cierre,10,300)
cv2.imshow('Canny',Bordes)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
umbr=cv2.adaptiveThreshold(gris,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,12)
cv2.imshow('Umbralizacion',umbr)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
dilatacion=cv2.dilate(umbr,Kernel2,iterations=1)
# dilatacion=cv2.dilate(dilatacion,Kernel,iterations=1)

cv2.imshow('dilatacion',dilatacion)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
gradiente=cv2.morphologyEx(umbr,cv2.MORPH_GRADIENT,Kernel2)
cv2.imshow('gradiente',gradiente)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
img2 = cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
contours,hierachy=cv2.findContours(umbr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2 ,contours, -1, (0,0,255),2)
cv2.imshow('imagen',img2)
cv2.imshow('gradiente',gradiente)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
img2 = cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
ht, wt = img2.shape[:2]
AreaImg=ht*wt
mask=np.ones((ht,wt),np.uint8)*0#Solo es para ver los cortonos
contours1,hierachy1=cv2.findContours(umbr,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#Contours1 es borde externo, contours es borde de rectangulos
cv2.drawContours(img2 ,contours,-1, (0,0,255) , 2)

result=cv2.drawContours(mask ,contours,-1, 255 , 2)
result1=cv2.drawContours(mask.copy() ,contours1,-1, 255 , 2)


cv2.imshow('imagen',img2)
cv2.imshow('gradiente',gradiente)
cv2.imshow('resultado',result)
cv2.imshow('resultado1',result1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [18]:
ret, th = cv2.threshold(result, 100, 255, cv2.THRESH_BINARY_INV)
mask = np.zeros((ht+2, wt+2), np.uint8)
cv2.floodFill(th, mask,(0,0),255); # position = (200,200)
out = cv2.bitwise_not(th)
kernel = np.ones((7,7),np.uint8)
out=cv2.morphologyEx(out,cv2.MORPH_OPEN,cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
#out=cv2.morphologyEx(out,cv2.MORPH_DILATE,cv2.getStructuringElement(cv2.MORPH_RECT,(5,5)))
#out= cv2.dilate(out,kernel,iterations = 5)
#out=cv2.bitwise_not(out)
outres = cv2.resize(out, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
cv2.imshow('out',outres)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [19]:
cnt, h = cv2.findContours(out,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
imageres=cv2.resize(image, (int(height/6), int(width/6)),interpolation=cv2.INTER_AREA )
rectangulos=[]
for i in range(len(cnt)):
            area = cv2.contourArea(cnt[i])
            if(area>(AreaImg/18) and area<(AreaImg)):
                  mask1 = np.zeros_like(img2)
                  cv2.drawContours(mask1, cnt, i, 255,-1)
                  x,y,w,h = cv2.boundingRect(cnt[i])
                  rec=[x,y,w,h]
                  rectangulos.append(rec)
                  rectangulos.sort(key=lambda rectangulo: rectangulo[0])
                  crop= imageres[ y:h+y,x:w+x]
                  cv2.imshow("snip",crop)
                  if(cv2.waitKey(0))==27:break                
cv2.destroyAllWindows()
     
    

In [20]:
#Se separan los rectangulos para ordenarlos, anteriormente se ordenaron de menor a mayor en x encontrando los que están más a la izquierda y ahora en se
#se centrará en encontrar de abajo hacía arriba
#ESTE ORDEN SOLO SIRVE PARA TARJETON CON 9 POSIBILIDADES
izquierda=rectangulos[0:3]
medio=rectangulos[3:6]
derecha=rectangulos[6:9]

izquierda.sort(key=lambda rectangulo: rectangulo[1])
medio.sort(key=lambda rectangulo: rectangulo[1])
derecha.sort(key=lambda rectangulo: rectangulo[1])

candidato=[]
for v in range(len(izquierda)):
      x=izquierda[v][0]
      y=izquierda[v][1]
      w=izquierda[v][2]
      h=izquierda[v][3]
      crop=imageres[y:h+y,x:w+x]
      candidato.append(crop)
      x=medio[v][0]     #FUNCIONA PORQUE IZAUEIRDA MEDIO Y DERECHA TIENEN LA MISMA LONGITUD LEE EL TARJETON DE IZQUIERDA A DERECHA DE ARRIBA ABAJO
      y=medio[v][1]
      w=medio[v][2]
      h=medio[v][3]
      crop=imageres[y:h+y,x:w+x]
      candidato.append(crop)
      x=derecha[v][0]
      y=derecha[v][1]
      w=derecha[v][2]
      h=derecha[v][3]
      crop=imageres[y:h+y,x:w+x]
      candidato.append(crop)

cv2.imshow('CANDIDATO',candidato[2])
cv2.waitKey(0)
cv2.destroyAllWindows()




In [21]:
#Extraccion de características
prueba =candidato[1]
prueba1=cv2.cvtColor(prueba,cv2.COLOR_RGB2GRAY)
#prueba1=cv2.equalizeHist(prueba1)
#prueba1=cv2.dilate(prueba1,Kernel2,iterations=1)
#prueba1=cv2.erode(prueba1,Kernel2,iterations=3)
# prueba1=cv2.dilate(prueba1,Kernel2,iterations=1)
#ret,prueba1=cv2.threshold(prueba1,100,255,cv2.THRESH_TRUNC)
salida=cv2.adaptiveThreshold(prueba1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,15)
#salida=cv2.Canny(prueba1,120,500)
salida=cv2.dilate(salida,Kernel2,iterations=2)
cv2.imshow('ORIGINAL',prueba)
cv2.imshow('PRE',prueba1)
cv2.imshow('FINAL',salida)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
final= prueba.copy()
lines = cv2.HoughLines(salida,1,np.pi/180,200)
y=0
if lines is None:
    y=False
else:
    print(len(lines))   
    for line in lines:
        rho,theta = line[0]
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        if(x2==x1):
            y=False
        else:
            m=(y2-y1)/(x2-x1)
            m= abs(m)
        if m<10:
            y=True
        cv2.line(final,(x1,y1),(x2,y2),(0,0,255),2)
if y== True:
    print("este es el voto")
else:
    print("este no es el candidato")
    
cv2.imshow('salida',final)
cv2.waitKey(0)
cv2.destroyAllWindows()

este no es el candidato


In [23]:
##Deteccion de la característica final
t=0
for contorno in candidato:
    print("el numero del candidato es el " + str(t))
    prueba =candidato[t]
    prueba1=cv2.cvtColor(prueba,cv2.COLOR_RGB2GRAY)
    salida=cv2.adaptiveThreshold(prueba1,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,17)
    salida=cv2.dilate(salida,Kernel2,iterations=2)
    cv2.imshow('ORIGINAL',prueba)
    cv2.imshow('PRE',prueba1)
    cv2.imshow('FINAL',salida)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    t=t+1
    final= prueba.copy()
    lines = cv2.HoughLines(salida,1,np.pi/180,200)
    y=0
    if lines is None:
        y= False
    else:   
        for line in lines:
            rho,theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a*rho
            y0 = b*rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            if(x2==x1):
                y=False
            else:
                m=(y2-y1)/(x2-x1)
                m= abs(m)
            if m<2:
                y=True
            cv2.line(final,(x1,y1),(x2,y2),(0,0,255),2)
    if y== True:
        print("este es el voto")
    else:
        print("este no es el candidato")

    cv2.imshow('salida',final)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

el numero del candidato es el 0
este no es el candidato
el numero del candidato es el 1
este no es el candidato
el numero del candidato es el 2
este no es el candidato
el numero del candidato es el 3
este no es el candidato
el numero del candidato es el 4
este no es el candidato
el numero del candidato es el 5
este es el voto
el numero del candidato es el 6
este no es el candidato
el numero del candidato es el 7
este no es el candidato
el numero del candidato es el 8
este no es el candidato
